In [1]:
## Installation required when using Google Colab
!pip install webcolors
!pip install word2number
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [0]:
## Imports
import spacy
from spacy import displacy
import webcolors
from word2number import w2n
import json

In [0]:
## Object to be used in Sketching module. Contains the necessary details 
# Find another name?
class Object:
  def __init__(self, name, color="white", size=1, number = 1, location="random"):
    self.name = name
    self.color = color
    self.size = size
    self.number = number
    self.location = location
  
  def print(self):
    print("Name: ",self.name,"\tColor:",self.color," Size:",self.size," Number:",self.number," Location:",self.location)

  def to_json(self):
    return json.dumps(self.__dict__)

In [0]:
## Functions to identify what information a feature gives about the object
def isColor(feature):
  return feature in webcolors.CSS3_NAMES_TO_HEX

def isNumber(feature):
  try:
    w2n.word_to_num(feature)
    return True
  except:
    return False

In [0]:
## Find the main object
def main_object(doc):
  obj = doc[0]
  for token in doc:
    if(token.dep_ == "attr"):
      obj = token
  return obj

## Extract the relevant information of the main object
# Implement better ->
def extract_features(doc, main_obj_token):
  ## Object of interest, dependency tag = "attr"
  # obj = [token for token in doc if (token.dep_ == "attr")]
  # print(type(obj))

  stack = []
  feature_lst = []
  stack=([child for child in main_obj_token.children])

  while stack:
    token = stack.pop()
    feature_lst.append(token.lemma_)
    if(token.pos_ != "NOUN"):
      stack+=[child for child in token.children]
  return feature_lst

## Creates an "Object" based on the given information
def match_features(feature_lst,main_obj_token):
  ob = Object(name = main_obj_token.text)
  for feature in feature_lst:
    if(isColor(feature)):
      ob.color = feature
    elif(isNumber(feature)):
      number = w2n.word_to_num(feature)
      ob.number = number
  return ob

In [0]:
def sentence_processing(sentence):
  doc = nlp(sentence)
  main_obj_token = main_object(doc)
  obj = match_features(extract_features(doc, main_obj_token), main_obj_token)

  obj.print()
  return obj.to_json()

In [0]:
## MAIN ##

nlp = spacy.load('en_core_web_sm')

# Get a sentence input from the user - to be deleted when merged with Speech module
sentence = input("Enter a sentence: ")

# Store the sentence as a Spacy Doc object. Contains information on the tokens
doc = nlp(sentence)
main_obj_token = main_object(doc)
obj = match_features(extract_features(doc, main_obj_token), main_obj_token)

obj.print()
obj.to_json()

In [0]:
## Some functions to help analyzing the sentence

# Lemma: Base form of the token, with no inflectional suffixes
# Part of Speech: Coarse-grained part-of-speech
# Tag: Fine-grained part-of-speech
# Dep: Syntactic dependency relation

def display_tokens(doc):
  start = '\033[36m'
  end = "\033[0;0m"
  print(start.ljust(45),"Lemma PoS Tag Dep",end)

  for token in doc:
    print(token.text.ljust(40) ,token.lemma_, token.pos_, token.tag_, token.dep_.ljust(30),"\t",spacy.explain(token.dep_))

def analyze_dependency(doc):
  for token in doc: 
    print("Token:",token,"\n\tAncestors: ",[ancestor for ancestor in token.ancestors],"\n\tChildren: ",[child for child in token.children])
    print("-------------------")

def dependency_tree(doc):
  return displacy.render(doc, jupyter=True, style='dep')

In [0]:
display_tokens(doc)